In [25]:
import datasets
import utils
import gmm_impute
import gmm_dae
import pickle_utils as pu
import numpy as np
import pandas as pd
import itertools as it
import missing_bayesian_mixture as mbm
import os
import denoising_ae
import category_dae
import tensorflow as tf
import mice_RF
import missForest

from sklearn.mixture import BayesianGaussianMixture

import imp
imp.reload(utils)
imp.reload(datasets)
imp.reload(mbm)
imp.reload(category_dae)
imp.reload(missForest)
imp.reload(denoising_ae)

<module 'denoising_ae' from '/home/adria/MIMIC/ts-baselines/timeless-imputation/denoising_ae.py'>

In [4]:
from IPython.display import HTML
def df_view(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))

In [39]:
dsets = datasets.datasets()
baseline = datasets.benchmark({#'MICE': datasets.memoize(utils.impute_mice),
                               'MissForest': datasets.memoize(utils.impute_missforest),
                               'MF_Py_par': datasets.memoize(lambda d, full_data: missForest.impute(
                                    d, full_data, sequential=False, print_progress=True)),
                               'MF_Py_seq': datasets.memoize(lambda d, full_data: missForest.impute(
                                    d, full_data, sequential=True, print_progress=True)),
                               'MF_prev_nomarg': datasets.memoize(lambda d, full_data: impute(
                                    d, full_data, sequential=True, print_progress=True,
                                    use_previous_prediction=True)),
                               'GP_vanilla_300': datasets.memoize(lambda d, full_data: None),
                              }, dsets, do_not_compute=True)

+++ Importing BreastCancer
+++ Importing Soybean
+++ Importing Ionosphere
V2 ; must have more than 1 possible value
+++ Importing BostonHousing
+++ Importing Servo
+++ Importing Shuttle
+++ Importing LetterRecognition


In [41]:
baseline.loc["mean_std"].drop(["LetterRecognition", "Shuttle"], axis=0)

NRMSE                      \
                             GP_vanilla_300 MF_Py_par MF_Py_seq   
BostonHousing MCAR_rows  0.1            NaN  0.186514  0.174660   
                         0.3            NaN  0.211605  0.213727   
                         0.5            NaN  0.256398  0.256856   
                         0.7            NaN  0.353217  0.337080   
                         0.9       0.384270  0.395895  0.422744   
              MCAR_total 0.1            NaN  0.145375  0.149661   
                         0.3       0.361165  0.167984  0.177417   
                         0.5       0.382446  0.205033  0.219206   
                         0.7       0.383924  0.249248  0.278603   
                         0.9            NaN  0.371953  0.367246   
BreastCancer  MCAR_rows  0.1            NaN       NaN       NaN   
                         0.3            NaN       NaN       NaN   
                         0.5            NaN       NaN       NaN   
                         0.7            NaN       NaN       NaN   
                         0.9            NaN       NaN       NaN   
              MCAR_total 0.1            NaN       NaN       NaN   
                         0.3            NaN       NaN       NaN   
                         0.5            NaN       NaN       NaN   
                         0.7            NaN       NaN       NaN   
                         0.9            NaN       NaN       NaN   
Ionosphere    MCAR_rows  0.1            NaN  0.644455  0.647443   
                         0.3            NaN  0.654825  0.658953   
                         0.5            NaN  0.734313  0.743048   
                         0.7            NaN  0.797926  0.812203   
                         0.9            NaN  0.981982  1.019554   
              MCAR_total 0.1            NaN  0.639807  0.633964   
                         0.3            NaN  0.651471  0.655008   
                         0.5            NaN  0.710881  0.704628   
                         0.7            NaN  0.769881  0.773842   
                         0.9            NaN  0.981797  1.047983   
Servo         MCAR_rows  0.1            NaN       NaN       NaN   
                         0.3            NaN       NaN       NaN   
                         0.5            NaN       NaN       NaN   
                         0.7            NaN       NaN       NaN   
                         0.9            NaN       NaN       NaN   
              MCAR_total 0.1            NaN       NaN       NaN   
                         0.3            NaN       NaN       NaN   
                         0.5            NaN       NaN       NaN   
                         0.7            NaN       NaN       NaN   
                         0.9            NaN       NaN       NaN   
Soybean       MCAR_rows  0.1            NaN       NaN       NaN   
                         0.3            NaN       NaN       NaN   
                         0.5            NaN       NaN       NaN   
                         0.7            NaN       NaN       NaN   
                         0.9            NaN       NaN       NaN   
              MCAR_total 0.1            NaN       NaN       NaN   
                         0.3            NaN       NaN       NaN   
                         0.5            NaN       NaN       NaN   
                         0.7            NaN       NaN       NaN   
                         0.9            NaN       NaN       NaN   

                                                                  PFC  \
                             MF_prev_nomarg MissForest GP_vanilla_300   
BostonHousing MCAR_rows  0.1       0.208735   0.167288            NaN   
                         0.3       0.218386   0.213626            NaN   
                         0.5       0.257683   0.236989            NaN   
                         0.7       0.341918   0.340666            NaN   
                         0.9       0.421209   0.393561       0.068282   
              MCAR_total 0.1       0.17152

In [170]:
imp.reload(mbm)
bayesgmm = datasets.benchmark({'BGMM_50': (lambda p, d, full_data: mbm.impute_bayes_gmm(
    p, d, full_data=full_data, number_imputations=100, n_components=50))
                              }, dsets, do_not_compute=True)

In [295]:
results = pd.concat([baseline, bayesgmm, nnresults], axis=1).loc['mean_std']
results = results.drop([('total_cats', 'MissForest'), ('total_cats', 'MLP_2x256_SGD')], axis=1)
results.sort_index(level=[0,1], inplace=True, axis=1)
results

In [319]:
# Export the results to LaTeX

s = (("""\\documentclass[12pt,a3paper,landscape,oneside]{article}
\\usepackage[utf8]{inputenc}
\\usepackage[english]{babel}
\\usepackage{setspace}
\\usepackage{parskip}
\\usepackage{color}
\\usepackage{amssymb,amsmath,amsfonts}
\\usepackage{graphicx}
\\usepackage{hyperref}
\\hypersetup{colorlinks,breaklinks,urlcolor=blue,linkcolor=blue} % Set link colors throughout the document
\\usepackage[left=2cm,right=2cm,top=2cm,bottom=2cm]{geometry}
\\usepackage{enumitem}   
\\usepackage{import}
\\usepackage{multicol}

\\usepackage{csquotes}
\\begin{document}\n""" + results.to_latex() + "\n\\end{document}")
     .replace("\\toprule", "\\hline")
     .replace("\\bottomrule", "\\hline")
     .replace("\\midrule", "\\hline"))

with open("measurements.tex", "w") as f:
    f.write(s)


In [ ]:
from rpy2.robjects.packages import importr
import rpy2.robjects
import rpy2.robjects.pandas2ri
import rpy2.robjects.numpy2ri
import rpy2.rinterface
import numpy as np
import pickle_utils as pu
import pandas as pd
import collections
import utils
import os
import collections
import unittest

rpy2.robjects.pandas2ri.activate()
rpy2.robjects.numpy2ri.activate()
py2ri = rpy2.robjects.pandas2ri.py2ri
ri2py = rpy2.robjects.pandas2ri.ri2py
R = rpy2.robjects.r

base = importr("base")
R_utils = importr("utils")
mlbench = importr("mlbench")
randomForest = importr("randomForest")

In [77]:
dsets_label = datasets.datasets(exclude_labels=False)
path = "impute_benchmark"
for algo_name in ["BayesGMM", "MICE", "MissForest"]:
    for data_name, (full_data, cat_keys) in dsets_label.items():
        if data_name == 'BostonHousing':
            continue

        for ampute_fun_name in ['MCAR_total', 'MCAR_rows']:
            for proportion in [.1, .3, .5, .7, .9]:
                for norm_type in ['mean_std']:  # 'min_max']:
                    amputed_name = '{:s}_{:s}_{:.1f}'.format(
                        data_name, ampute_fun_name, proportion) 
                    imputed_path = os.path.join(
                        path, 'imputed_{:s}_{:s}.pkl.gz'.format(
                            algo_name, amputed_name))
                    if not os.path.exists(imputed_path):
                        continue
                    amputed_data = pu.load(os.path.join(path, 'amputed_'+amputed_name+'.pkl.gz'))
                    imputed_data = pu.load(imputed_path)
                    (_ad, _id), moments = utils.normalise_dataframes(
                        amputed_data, imputed_data, method=norm_type)
                    
                    to_learn = pd.concat([_id, full_data[["Class"]]], axis=1)
                    test_mask = np.random.rand(len(to_learn)) < 0.2
                    training_set = to_learn[~test_mask]
                    test_set = to_learn[test_mask]
                    
                    


+++ Importing Ionosphere
V2 ; must have more than 1 possible value
+++ Importing Soybean
+++ Importing BreastCancer
+++ Importing Servo
+++ Importing BostonHousing


AssertionError: All dataframes must have the same columns